<a href="https://colab.research.google.com/github/marcin192/corona/blob/master/Corona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
#%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
plt.rcParams['figure.figsize'] = [15, 5]
from IPython import display
from ipywidgets import interact, widgets
pd.set_option('mode.chained_assignment', None)

In [0]:
def download(option):#funkcja pobierająca pliki .csv z repozytorium GitHub w zależności od wybranej opcji pobiera albo dane o koronawirusie albo o populacji i zwraca DataFrame
  if option=="Cases":
    url = f"https://raw.githubusercontent.com/marcin192/pop/master/states/time_series_covid19_confirmed_global.csv"
  elif option=="Population":
    url=f"https://raw.githubusercontent.com/marcin192/pop/master/popek.csv"
  dfd = pd.read_csv(url, error_bad_lines=False)
  return dfd

In [0]:
def dateTostr(date):#funkcja konwertujące datę z datetime na string 
  date_str=date.strftime('%m/%d/%y').lstrip("0").replace(" 0", " ").replace(" 0/"," /")
  return date_str

In [0]:
def differentiate(datex,dt):#funkcja wyznacza średni dzienny przyrost pzrypadków dla podanej daty "datex" i okresu poprzedzającego "dt"
  datexC=dateTostr(datex)#konwersja z obiektu datetime daty kończącej okres do stringa, potrzebne do obsługi DataFrame
  dateo=datex+datetime.timedelta(days=-dt)#wyznaczenie daty rozpoczynającej okres
  dateoC=dateTostr(dateo)#konwersja z obiektu datetime daty rozpoczynającej okres do stringa, potrzebne do obsługi DataFrame
  dfC=download("Cases")#pobranie DataFrame z przypadkami COVID-19
  dfM=dfC[["Country/Region",datexC,dateoC]]#wybranie interesujących nas kolumn
  dfM[["diff"]]=-dfM[[dateoC]]#stworzenie nowej kolumny w której będzie przyrost
  dfM[["diff"]]=0
  it=0#zmienna indeksująca
  l=dfM.shape[0]#ilośc pozycji tabeli
  for i in range(0,l):
    dfM["diff"].iloc[it:it+1]=(dfM[datexC].iloc[it:it+1]-dfM[dateoC].iloc[it:it+1])/dt#wyznaczenie średniego przyrostu poprzez różnicę miedzy danymi dniami podzieloną przez liczbe dni okresu
    it=it+1
  
  return dfM.sort_values(by="diff",ascending=False)#zwrócenie DataFrame posortowanego malejąco po przyrostach


In [0]:
def plotd(date,dt,ex):#funkcja wyświetlająca kartogram średnich dziennych przyrostów dla danej daty i okresu poprzedzającego z możliwością podniesienia wartości do "ex" potęgi
                                                                                                  #dla "ex" od 0 do 1 pomaga zobrazować wyniki,które są bardzo zróżnicowane
  dfg=differentiate(date,dt)#DataFrame zawierający średnie dzienne przyrosty 
  dfg=dfg.groupby(["Country/Region"]).sum()#pogrupowanie względem kraju i zsumowanie wyników każdej prowincji danego kraju
  formated_gdf = dfg.reset_index()#przywrócenie indeksowania by było ciagłe
  formated_gdf['size'] = formated_gdf[["diff"]].pow(ex)#dodanie nowej kolumny z wartośćiami dla kartogramu przeskalowanymi potęgą do "ex"



  fig = px.choropleth(formated_gdf, locations="Country/Region", locationmode='country names', #tworzenie kartogamu
                     color="diff", hover_name="Country/Region", 
                     range_color= [0, max(formated_gdf["diff"].pow(ex))+2], 
                     projection="natural earth",
                     labels={"diff":"Średni dzienny przyrost"},
                     title=f"Średni dzienny przyrost w okresie od {date+datetime.timedelta(days=-dt)} do {date}")





  fig.show()#wyświetlenie wykresu
  return formated_gdf.sort_values(by="diff",ascending=False).reset_index().drop(["index"],axis=1)#zwrócenie DataFrame posortowanego względem przyrostu z zresetowanym indeksowaniem

In [0]:
def population(option):#funckja zwracająca DataFrame populacji z posortowanymi alfabetycznie państwami w zależności od wybranej opcji 
                                                                                              #1 zwraca DataFrame tylko z populacja
                                                                                              #inne zwraca DataFrame z populacją i odpowiadającym jej państwom
  pop=download("Population")
  pop=pop.sort_values(by="Country")#sortowanie alfabetycznie po państwach
  pop=pop.reset_index()#zresetowanie indeksowania
  pop=pop.drop(["index"],axis=1)
  if(option==1):
    pop=pop.drop(["Country"],axis=1)#usunięcie kolumny z panstwami dla "option" równej 1
    return pop
  else:
    return pop

In [0]:
def confirmparse():#funkcja dopasowująca DataFrame przypadków dla danych populacji
  dfC=download("Cases")#pobranie DataFrame z przypadkami
  dfC=dfC.groupby(by="Country/Region").sum().reset_index()#scalenie wyników względem kraju
  dfC=dfC.drop(dfC[dfC["Country/Region"]=="Kosovo"].index,axis=0)
  dfC=dfC.drop(dfC[dfC["Country/Region"]=="Diamond Princess"].index,axis=0)#usuniecie pozycji dla których nie są dostępne dane o populacji
  dfC=dfC.drop(dfC[dfC["Country/Region"]=="West Bank and Gaza"].index,axis=0)
  dfC=dfC.reset_index()#zresetowanie indeksowania
  dfC=dfC.drop(["index"],axis=1)
  index1=dfC[dfC["Country/Region"]=="Congo (Brazzaville)"].index
  index2=dfC[dfC["Country/Region"]=="Congo (Kinshasa)"].index #scalenie dwóch części państwa do jednej
  dfC.loc[index1[0]] += dfC.loc[index2[0]]#przekierowanie danych do jednego wiersza
  dfC=dfC.drop(dfC[dfC["Country/Region"]=="Congo (Kinshasa)"].index,axis=0)
  dfC.at[index1[0],"Country/Region"]="Congo"#przemianowanie częsci pańtwa na państwo
  dfC=dfC.drop(["Lat","Long"],axis=1)#pozbycie się nieużywanych kolumn
  dfC=dfC.reset_index()
  dfC=dfC.drop(["index"],axis=1)#zresetowanie indeksowania
  return dfC#zwrócenie DataFrame z przypadkami w dobrej formie

In [0]:
def differentiate_population(datex,dt):#funkcja wyznacza średni dzienny przyrost przypadków na 1 000 000 mieszkańców dla podanej daty "datex" i okresu poprzedzającego "dt"
  datexC=dateTostr(datex)#konwersja z obiektu datetime daty kończącej okres do stringa, potrzebne do obsługi DataFrame
  dateo=datex+datetime.timedelta(days=-dt)#data rozpoczynająca okres 
  dateoC=dateTostr(dateo)#konwersja z obiektu datetime daty rozpoczynajacej okres do stringa, potrzebne do obsługi DataFrame
  c=confirmparse()#przekazanie dobrze obrobionego DataFrame z przypadkami
  p=population(1)#przekazanie DataFrame z populacją zawierajacego tylko dane o populacji
  j=pd.concat([p,c],axis=1)#połączenie dwóch DataFrame w jeden
  m=j[["Population","Country/Region",datexC,dateoC]]#wybranie interesujących kolumn
  m[["diff%"]]=m[[dateoC]]#stworzenie nowej kolumny z przyrostem
  m[["diff%"]]=0
  it=0#zmienna indeksująca
  l=m.shape[0]#ile jest pozycji
  for i in range(0,l):
    m["diff%"].iloc[it:it+1]=(m[datexC].iloc[it:it+1]-m[dateoC].iloc[it:it+1])/dt/(m["Population"].iloc[it:it+1])*1000000#wyznaczenie przyrostu poprzez podzielenie różnicy przez okres
                                                                                                                        #i ilość populacji pąnstwa i pomnożenie przez milion
    it=it+1
  
  return m.sort_values(by="diff%",ascending=False)#zwrócenie posortowanego malejąco po przyrostach DataFrame

In [0]:
def plotdp(date,dt,ex):
  dfg=differentiate_population(date,dt)#DataFrame zawierający średnie dzienne przyrosty
  dfg=dfg.groupby(["Country/Region"]).sum()#pogrupowanie względem kraju i zsumowanie wyników każdej prowincji danego kraju
  #indexNames=dfg[dfg["Population"]<1000000].index#pozbycie sie państw poniżej 1 000 000 mieszkańców, by wyniki były sprawiedliwe
  #dfg.drop(indexNames , inplace=True)
  formated_gdf = dfg.reset_index()
  formated_gdf['size'] = formated_gdf[["diff%"]].pow(ex)#utworzenie nowej kolumny z przyrostami przeskalowanymi



  fig = px.choropleth(formated_gdf, locations="Country/Region", locationmode='country names', #tworzenie wykresu
                     color="diff%", hover_name="Country/Region", 
                     range_color= [0, max(formated_gdf["diff%"].pow(ex))+2], 
                     projection="natural earth",
                     labels={"diff":"Średni dzienny przyrost"},
                     title=f"Średni dzienny przyrost na 1 000 000 mieszkańców w okresie od {date+datetime.timedelta(days=-dt)} do {date}")





  fig.show()#wyświetlenie wykresu
  return formated_gdf.sort_values(by="diff%",ascending=False).reset_index().drop(["index"],axis=1)#zwrócenie DataFrame posortowanego względem przyrostu z zresetowanym indeksowaniem

In [35]:
plotdp(datetime.date(2020,3,25),7,0.5)

,Country/Region,Population,3/25/20,3/18/20,diff%,size
0,Denmark,988.0,1862,1116,107865.818392,328.429320
1,Italy,2961167.0,74386,35713,1865.721955,43.194004
2,Haiti,801.0,8,0,1426.787944,37.772847
3,France,2225734.0,25600,9124,1057.500261,32.519229
4,Greece,112523.0,821,418,511.641429,22.619492
...,...,...,...,...,...,...
167,Somalia,59308690.0,1,1,0.000000,0.000000
168,Saint Vincent and the Grenadines,33931.0,1,1,0.000000,0.000000
169,Maldives,20250833.0,13,13,0.000000,0.000000
170,Zimbabwe,0.0,3,0,0.000000,0.000000
